In [13]:
from scipy import ndimage
import shutil
from tqdm import tqdm
import cv2
from scipy import ndimage
import matplotlib.pyplot as plt
from PIL import Image
import os
from random import shuffle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
%matplotlib inline

## (kernel使うときに変更すべき変数)

In [3]:
padding_data_path = "/home/tidal/ML_Data/SIIM-ISIC_Melanoma_Classification/SIIM-ISIC-Melanoma-Classification/pad_jpg"
melanoma_external_malignant_256 = "/home/tidal/ML_Data/SIIM-ISIC_Melanoma_Classification/melanoma_external_malignant_256/"
SIIM_ISIC_Melanoma_Classification = "/home/tidal/ML_Data/SIIM-ISIC_Melanoma_Classification/SIIM-ISIC-Melanoma-Classification/"

## config

## resize & write

In [8]:
#画像のリサイズ
#入力：加工元のパス、サイズ(縦)、サイズ(横)
#出力：加工後の画像(mtx)、ImageName(mtx)
def resize_image(chdir, size_X, size_Y):
    #change directry
    os.chdir(chdir)
    rsz_file = os.listdir()
    X_data = []
    img_name = []
    for file in rsz_file:
        rsz_img = cv2.imread(file)
        rsz_img = cv2.resize(rsz_img, (size_X, size_Y) ) 
        X_data.append(rsz_img)
        img_name.append(file)
    print("end:resize_image")
    return X_data, img_name

#画像の書き込み
#入力：加工後の画像(mtx)、ImageName(mtx), テスト用/学習用(test/train)、保存フォルダ名
#出力：成功有無
def write_to_localfolder(img_mtx, img_name, test_or_train, folder_name):
    if test_or_train == "test":
        img_dir = SIIM_ISIC_Melanoma_Classification + "pad_jpg/test/" + folder_name
    elif test_or_train == "train":
        img_dir = SIIM_ISIC_Melanoma_Classification + "pad_jpg/train/" + folder_name
    else:
        print("3rd argument is (test/train).")
        print("end:write_to_localfolder")
        return False
    
    os.mkdir(img_dir)
    for ix in range(len(img_mtx)):
        cv2.imwrite(img_dir + '/{0}'.format(img_name[ix]), img_mtx[ix])
    
    print("end:write_to_localfolder")
    return True

#画像のリサイズ&書き込み(溜め込むサイズを小さくなるように改良)
#入力：加工元のパス、サイズ(縦)、サイズ(横)、テスト用/学習用(test/train)、保存フォルダ名
#出力：成功有無
def ResizeAndWriteImg(chdir, size_X, size_Y, test_or_train, folder_name):
    if test_or_train == "test":
        img_dir = SIIM_ISIC_Melanoma_Classification + "pad_jpg/test/" + folder_name
    elif test_or_train == "train":
        img_dir = SIIM_ISIC_Melanoma_Classification + "pad_jpg/train/" + folder_name
    else:
        print("3rd argument is (test/train).")
        print("end:write_to_localfolder")
        return False
    os.mkdir(img_dir)
    os.chdir(chdir)
    rsz_file = os.listdir()
    for file in rsz_file:
        rsz_img = cv2.imread(file)
        rsz_img = cv2.resize(rsz_img, (size_X, size_Y) ) 
        cv2.imwrite(img_dir + '/{0}'.format(file), rsz_img)
    print("end:resize_image")
    return True


#画像のリサイズ&書き込み(溜め込むサイズを小さくなるように改良)
#入力：加工元のパス、サイズ(縦)、サイズ(横)、テスト用/学習用(test/train)、保存フォルダ名
#出力：成功有無
def CroppingImg(chdir, size_X, size_Y, test_or_train, folder_name):
    if test_or_train == "test":
        img_dir = SIIM_ISIC_Melanoma_Classification + "pad_jpg/test/" + folder_name
    elif test_or_train == "train":
        img_dir = SIIM_ISIC_Melanoma_Classification + "pad_jpg/train/" + folder_name
    else:
        print("3rd argument is (test/train).")
        print("end:write_to_localfolder")
        return False
    
    if not os.path.exists(img_dir):
        os.mkdir(img_dir)
    
    os.chdir(chdir)
    rsz_file = os.listdir()
    for file in rsz_file:
        rsz_img = cv2.imread(file)
        w = rsz_img.shape[1]; h = rsz_img.shape[0]; s = min(w,h)
        w2 = (w-s)//2; h2 = (h-s)//2
        rsz_img = rsz_img[h2:h-h2,w2:w-w2,:]
        rsz_img = cv2.resize(rsz_img, (size_X, size_Y) ) 
        cv2.imwrite(img_dir + '/{0}'.format(file), rsz_img)
    print("end:resize_image")
    return True

## augment

In [22]:
#入力：加工元のパス、加工先のパス + folder_name
#出力：成功有無
def BGR2GRAY(chdir, img_dir):
    os.mkdir(img_dir)
    os.chdir(chdir)
    rsz_file = os.listdir()
    for file in rsz_file:
        rsz_img = cv2.imread(file)
        rsz_img = cv2.cvtColor(rsz_img, cv2.COLOR_BGR2GRAY)
        cv2.imwrite(img_dir + '/{0}'.format(file), rsz_img) 
    print("end:resize_image")
    return True

##入力：加工元のパス、加工先のパス + folder_name, 閾値
##出力：成功有無
#def threshold_TOZERO(chdir, img_dir, thr):
#    os.mkdir(img_dir)
#    os.chdir(chdir)
#    rsz_file = os.listdir()
#    for file in rsz_file:
#        rsz_img = cv2.imread(file)
#        rsz_img = cv2.threshold(rsz_img, thr, 255, cv2.THRESH_TOZERO)[1]
#        cv2.imwrite(img_dir + '/{0}'.format("THRESH_TOZERO_" + file), rsz_img) #加工した内容を名前に残す。
#    print("end:resize_image")
#    return True

#入力：加工元のパス、加工先のパス + folder_name, 角度
#出力：成功有無
def ROTATE(chdir, img_dir, ang):
    os.mkdir(img_dir)
    os.chdir(chdir)
    rsz_file = os.listdir()
    str_ang = str(ang)
    for file in rsz_file:
        rsz_img = cv2.imread(file)
        rsz_img = ndimage.rotate(rsz_img,ang)
        cv2.imwrite(img_dir + '/{0}'.format(file), rsz_img) 
    print("end:resize_image")
    return True

def shade_of_gray_cc(img, power=6, gamma=None):
    """
    img (numpy array): the original image with format of (h, w, c)
    power (int): the degree of norm, 6 is used in reference paper
    gamma (float): the value of gamma correction, 2.2 is used in reference paper
    """
    img_dtype = img.dtype

    if gamma is not None:
        img = img.astype('uint8')
        look_up_table = np.ones((256,1), dtype='uint8') * 0
        for i in range(256):
            look_up_table[i][0] = 255 * pow(i/255, 1/gamma)
        img = cv2.LUT(img, look_up_table)

    img = img.astype('float32')
    img_power = np.power(img, power)
    rgb_vec = np.power(np.mean(img_power, (0,1)), 1/power)
    rgb_norm = np.sqrt(np.sum(np.power(rgb_vec, 2.0)))
    rgb_vec = rgb_vec/rgb_norm
    rgb_vec = 1/(rgb_vec*np.sqrt(3))
    img = np.multiply(img, rgb_vec)

    # Andrew Anikin suggestion
    img = np.clip(img, a_min=0, a_max=255)
    
    return img.astype(img_dtype)

def apply_cc(imgfolder, output_folder_path, resize=None):
    
    if not os.path.isdir(output_folder_path):
        os.mkdir(output_folder_path)    
    
    imgNMs = os.listdir(imgfolder)
    
    with tqdm(total=len(imgNMs), ascii=True, ncols=100) as t:     
        for imgNM in imgNMs:
            img_ = cv2.imread(imgfolder + '/' + imgNM, cv2.IMREAD_COLOR)
            if resize is not None:
                img_ = cv2.resize(img_, resize, cv2.INTER_AREA)
            np_img = shade_of_gray_cc (img_)            
            cv2.imwrite(os.path.join(output_folder_path, imgNM.split('.')[0] + '.jpg'), np_img)
            t.update()

## Exec augmentation

In [6]:
%%time
SizeMtx = [64, 128, 256]
ang = 270
for size in SizeMtx:
    folder_name = str(size) + '_' + str(size)
    ROTATE(SIIM_ISIC_Melanoma_Classification + "pad_jpg/test/" + folder_name, 
            SIIM_ISIC_Melanoma_Classification + "pad_jpg/test_agm/" + folder_name + "_ROTATE_" + str(ang), ang)

#for size in SizeMtx:
#    folder_name = str(size) + '_' + str(size) + '_merge'
#    ROTATE(SIIM_ISIC_Melanoma_Classification + "pad_jpg/train/" + folder_name, 
#            SIIM_ISIC_Melanoma_Classification + "pad_jpg/train_agm/" + folder_name + "_ROTATE_" + str(ang), ang)

end:resize_image
end:resize_image
end:resize_image
CPU times: user 4min 42s, sys: 1.27 s, total: 4min 43s
Wall time: 4min 45s


## excute

In [8]:
%%time
SizeMtx = [64,96,128,160,192,256,288,384]
for size in SizeMtx:
    folder_name = str(size) + '_' + str(size) + '_malignant2019'
    ResizeAndWriteImg(SIIM_ISIC_Melanoma_Classification + "jpeg/train_malignant_2019/", size, size, "train", folder_name)

end:resize_image
end:resize_image
end:resize_image
end:resize_image
end:resize_image
end:resize_image
end:resize_image
end:resize_image
CPU times: user 45.6 s, sys: 4.68 s, total: 50.3 s
Wall time: 45.1 s


In [16]:
%%time
SizeMtx = [224]
for size in SizeMtx:
    folder_name = str(size) + '_' + str(size) + '_malignant2019'
    ResizeAndWriteImg(SIIM_ISIC_Melanoma_Classification + "jpeg/train_malignant_2019/", size, size, "train", folder_name)

end:resize_image
CPU times: user 5.2 s, sys: 542 ms, total: 5.74 s
Wall time: 5.82 s


In [9]:
%%time
SizeMtx = [64]
for size in SizeMtx:
    folder_name = str(size) + '_' + str(size) + '_CenteredCrop'
    CroppingImg(SIIM_ISIC_Melanoma_Classification + "jpeg/train_malignant_2019/", size, size, "train", folder_name)
    CroppingImg(SIIM_ISIC_Melanoma_Classification + "jpeg/train/", size, size, "train", folder_name)
    CroppingImg(SIIM_ISIC_Melanoma_Classification + "jpeg/test/", size, size, "test", folder_name)

end:resize_image
end:resize_image
end:resize_image
CPU times: user 57min 29s, sys: 5min 59s, total: 1h 3min 28s
Wall time: 1h 3min 34s


In [12]:
%%time
SizeMtx = [64, 224, 256, 288, 384, 448, 512]
for size in SizeMtx:
    folder_name = str(size) + '_' + str(size) + '_CenteredCrop'
    CroppingImg(SIIM_ISIC_Melanoma_Classification + "jpeg/train_malignant_2019/", size, size, "train", folder_name)
    CroppingImg(SIIM_ISIC_Melanoma_Classification + "jpeg/train/", size, size, "train", folder_name)
    CroppingImg(SIIM_ISIC_Melanoma_Classification + "jpeg/test/", size, size, "test", folder_name)

end:resize_image
end:resize_image
end:resize_image
end:resize_image
end:resize_image
end:resize_image
end:resize_image
end:resize_image
end:resize_image
end:resize_image
end:resize_image
end:resize_image
end:resize_image
end:resize_image
end:resize_image
end:resize_image
end:resize_image
end:resize_image
end:resize_image
end:resize_image
end:resize_image
CPU times: user 7h 14min 9s, sys: 47min 55s, total: 8h 2min 5s
Wall time: 7h 41min 33s


In [23]:
%%time
SizeMtx = [64, 224, 256, 288, 384, 448, 512]
pad_jpg = SIIM_ISIC_Melanoma_Classification + "pad_jpg/"
for size in SizeMtx:
    in_folder  = str(size) + '_' + str(size) + '_CenteredCrop'
    out_folder = str(size) + '_' + str(size) + '_GrayColorConstancy'
    apply_cc(pad_jpg + "train/" + in_folder, pad_jpg + "train/" + out_folder)
    apply_cc(pad_jpg + "test/" + in_folder, pad_jpg + "test/" + out_folder)

100%|#########################################################| 10982/10982 [03:02<00:00, 60.31it/s]

CPU times: user 38min 23s, sys: 23 s, total: 38min 46s
Wall time: 39min 4s


In [21]:
DIR = pad_jpg + "train/" + in_folder
rsz_file = os.listdir(DIR)
rsz_file

['ISIC_8024261.jpg',
 'ISIC_5556498.jpg',
 'ISIC_6306333.jpg',
 'ISIC_1342739.jpg',
 'ISIC_0000446.jpg',
 'ISIC_3123711.jpg',
 'ISIC_9631972.jpg',
 'ISIC_0908367.jpg',
 'ISIC_4910094.jpg',
 'ISIC_4419720.jpg',
 'ISIC_5359353.jpg',
 'ISIC_0772957.jpg',
 'ISIC_4349418.jpg',
 'ISIC_4043580.jpg',
 'ISIC_9460903.jpg',
 'ISIC_2801761.jpg',
 'ISIC_2974135.jpg',
 'ISIC_6963859.jpg',
 'ISIC_0943299.jpg',
 'ISIC_7733258.jpg',
 'ISIC_8387944.jpg',
 'ISIC_9683184.jpg',
 'ISIC_4342416.jpg',
 'ISIC_0581628.jpg',
 'ISIC_6384912.jpg',
 'ISIC_0770861.jpg',
 'ISIC_3003666.jpg',
 'ISIC_7859801.jpg',
 'ISIC_1534800.jpg',
 'ISIC_1977110.jpg',
 'ISIC_7907495.jpg',
 'ISIC_5315726.jpg',
 'ISIC_8904639.jpg',
 'ISIC_1639765.jpg',
 'ISIC_3481509.jpg',
 'ISIC_8709649.jpg',
 'ISIC_4476854.jpg',
 'ISIC_1276130.jpg',
 'ISIC_1258986.jpg',
 'ISIC_8212937.jpg',
 'ISIC_3443234.jpg',
 'ISIC_8716192.jpg',
 'ISIC_1834416.jpg',
 'ISIC_6239639.jpg',
 'ISIC_0894136.jpg',
 'ISIC_1888148.jpg',
 'ISIC_2157103.jpg',
 'ISIC_919996

## trainとmeta validationの分割

In [9]:
#config
METAVALSIZE = 8282 #train:meta val = 3:1
ImgNmMetaVal = pd.read_csv(SIIM_ISIC_Melanoma_Classification + 'pad_jpg/ImgNmMetaVal.csv')

In [10]:
ImgNmMetaVal

,Unnamed: 0,image_name
0,0,ISIC_0452656
1,1,ISIC_0507189
2,2,ISIC_1144286
3,3,ISIC_1199236
4,4,ISIC_1303205
...,...,...
8310,8310,ISIC_8609756
8311,8311,ISIC_8620064
8312,8312,ISIC_9212054
8313,8313,ISIC_9659306


In [11]:
ImgNmMetaVal = pd.read_csv(SIIM_ISIC_Melanoma_Classification + 'pad_jpg/ImgNmMetaVal.csv')
ImgNmMetaVal["image_name"]

0       ISIC_0452656
1       ISIC_0507189
2       ISIC_1144286
3       ISIC_1199236
4       ISIC_1303205
            ...     
8310    ISIC_8609756
8311    ISIC_8620064
8312    ISIC_9212054
8313    ISIC_9659306
8314    ISIC_9725585
Name: image_name, Length: 8315, dtype: object

In [12]:
#画像の一部をmetavalフォルダに移動
#入力：保存フォルダ名
#出力：なし
def MvMetaVal(folder_name):
    train_dir = SIIM_ISIC_Melanoma_Classification + "pad_jpg/train/" + folder_name
    meta_dir = SIIM_ISIC_Melanoma_Classification + "pad_jpg/metaval/" + folder_name
    os.mkdir(meta_dir)
    ImgNmMetaVal = pd.read_csv(SIIM_ISIC_Melanoma_Classification + 'pad_jpg/ImgNmMetaVal.csv')
    for nm in ImgNmMetaVal["image_name"]:
        shutil.move(train_dir + '/' + nm + '.jpg', meta_dir + '/' + nm + '.jpg')

In [65]:
MvMetaVal("256_256")
MvMetaVal("384_384")
MvMetaVal("512_512")
MvMetaVal("768_768")

In [36]:
#MetaVal.csv
trainCsv = pd.read_csv(SIIM_ISIC_Melanoma_Classification + 'train.csv')
ImgNmMetaVal = pd.read_csv(SIIM_ISIC_Melanoma_Classification + 'pad_jpg/ImgNmMetaVal.csv')
flg = True
for nm in ImgNmMetaVal["image_name"]:
    if flg:
        trainCsvMeta = trainCsv[trainCsv.image_name == nm]
        flg = False
    else:
        trainCsvMeta = trainCsvMeta.append(trainCsv[trainCsv.image_name == nm])
    ix = trainCsv[trainCsv.image_name == nm].index.values[0]
    trainCsv = trainCsv.drop(ix, axis=0)
trainCsv = trainCsv.reset_index(drop=True)
trainCsvMeta = trainCsvMeta.reset_index(drop=True)

## 分割したmetavalを元のフォルダに戻す

In [4]:
#画像の一部をmetavalフォルダに移動
#入力：保存フォルダ名
#出力：なし
def MvMetaVal2Train(folder_name):
    train_dir = SIIM_ISIC_Melanoma_Classification + "pad_jpg/train/" + folder_name
    meta_dir = SIIM_ISIC_Melanoma_Classification + "pad_jpg/metaval/" + folder_name
    ImgNmMetaVal = pd.read_csv(SIIM_ISIC_Melanoma_Classification + 'pad_jpg/ImgNmMetaVal.csv')
    for nm in ImgNmMetaVal["image_name"]:
        shutil.move(meta_dir + '/' + nm + '.jpg', train_dir + '/' + nm + '.jpg')

In [6]:
MvMetaVal2Train("224_224")
MvMetaVal2Train("256_256")
MvMetaVal2Train("384_384")
MvMetaVal2Train("512_512")
MvMetaVal2Train("768_768")

## sample_sub_metaval_stacking.csv

In [7]:
ImgNmMetaVal = pd.read_csv(SIIM_ISIC_Melanoma_Classification + 'train.csv')

In [8]:
ImgNmMetaVal

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0
...,...,...,...,...,...,...,...,...
33121,ISIC_9999134,IP_6526534,male,50.0,torso,unknown,benign,0
33122,ISIC_9999320,IP_3650745,male,65.0,torso,unknown,benign,0
33123,ISIC_9999515,IP_2026598,male,20.0,lower extremity,unknown,benign,0
33124,ISIC_9999666,IP_7702038,male,50.0,lower extremity,unknown,benign,0


In [9]:
ImgNmMetaVal = ImgNmMetaVal.loc[:, ['image_name', 'target']]
ImgNmMetaVal

,image_name,target
0,ISIC_2637011,0
1,ISIC_0015719,0
2,ISIC_0052212,0
3,ISIC_0068279,0
4,ISIC_0074268,0
...,...,...
33121,ISIC_9999134,0
33122,ISIC_9999320,0
33123,ISIC_9999515,0
33124,ISIC_9999666,0


In [10]:
ImgNmMetaVal[ImgNmMetaVal.target == 1]

,image_name,target
91,ISIC_0149568,1
235,ISIC_0188432,1
314,ISIC_0207268,1
399,ISIC_0232101,1
459,ISIC_0247330,1
...,...,...
32969,ISIC_9955163,1
33000,ISIC_9963177,1
33014,ISIC_9967383,1
33050,ISIC_9978107,1


In [11]:
ImgNmMetaVal.loc[:, 'target'] = 0

In [12]:
ImgNmMetaVal[ImgNmMetaVal.target == 1]

,image_name,target


In [13]:
ImgNmMetaVal[ImgNmMetaVal.target == 0]

,image_name,target
0,ISIC_2637011,0
1,ISIC_0015719,0
2,ISIC_0052212,0
3,ISIC_0068279,0
4,ISIC_0074268,0
...,...,...
33121,ISIC_9999134,0
33122,ISIC_9999320,0
33123,ISIC_9999515,0
33124,ISIC_9999666,0


In [14]:
ImgNmMetaVal.to_csv(SIIM_ISIC_Melanoma_Classification + 'sample_sub_metaval_stacking.csv')

## trainとmalignantのマージ

In [14]:
def MergeTrainAndMalignant(TrDir, MlgDir):
    shutil.copytree(TrDir, TrDir + '_merge')
    MlgDir_fileNm = os.listdir(MlgDir)
    for nm in MlgDir_fileNm:
        shutil.copyfile(MlgDir + "/" + nm, TrDir + '_merge' + "/" + nm)
    print("end.")

In [15]:
size = "128_128"
DIR = SIIM_ISIC_Melanoma_Classification + 'pad_jpg/train/' + size
MergeTrainAndMalignant(DIR, DIR+'_malignant2019')

end.


In [18]:
sizeMtx = ["160_160", "192_192", "224_224", "256_256", "288_288", "384_384"]
for size in sizeMtx:
    DIR = SIIM_ISIC_Melanoma_Classification + 'pad_jpg/train/' + size
    MergeTrainAndMalignant(DIR, DIR+'_malignant2019')

end.
end.
end.
end.
end.
end.


In [43]:
train_malig = pd.read_csv(SIIM_ISIC_Melanoma_Classification + "train_malig_2.csv")
train_malig = train_malig[train_malig.tfrecord != -1]
train_malig = train_malig.drop(['diagnosis', 'benign_malignant', 'width', 'height', 'source', 'ext'], axis=1)
train_malig = train_malig.reset_index(drop=True)
tfrecord_malig = train_malig.loc[:,['tfrecord']]
tfrecord_malig = tfrecord_malig-15
train_malig = train_malig.drop(['tfrecord'], axis=1)
train_malig

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,target
0,ISIC_0000070,-1,male,25.0,anterior torso,1
1,ISIC_0000076,-1,male,55.0,posterior torso,1
2,ISIC_0000144,-1,NaN,NaN,NaN,1
3,ISIC_0000158,-1,female,30.0,upper extremity,1
4,ISIC_0000284,-1,male,80.0,head/neck,1
...,...,...,...,...,...,...
575,ISIC_0024302,-1,female,25.0,NaN,1
576,ISIC_0024303,-1,male,60.0,NaN,1
577,ISIC_0024304,-1,male,70.0,NaN,1
578,ISIC_0024305,-1,male,85.0,NaN,1


In [44]:
tfrecord_malig

,tfrecord
0,0
1,0
2,0
3,0
4,0
...,...
575,14
576,14
577,14
578,14


In [39]:
train_df = pd.read_csv(SIIM_ISIC_Melanoma_Classification + 'train.csv')
train_df = train_df.drop(['diagnosis','benign_malignant'], axis=1)
train_df = train_df[train_df.tfrecord != -1]
train_df = train_df.drop(['width','height','patient_code'], axis=1)

train_df = train_df.reset_index(drop=True)
tfrecord = train_df.loc[:,['tfrecord']]
train_df = train_df.drop(['tfrecord'], axis=1)

test_df = pd.read_csv(SIIM_ISIC_Melanoma_Classification + 'test.csv')
train_df

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,0
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,0
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,0
...,...,...,...,...,...,...
32687,ISIC_9999134,IP_6526534,male,50.0,torso,0
32688,ISIC_9999320,IP_3650745,male,65.0,torso,0
32689,ISIC_9999515,IP_2026598,male,20.0,lower extremity,0
32690,ISIC_9999666,IP_7702038,male,50.0,lower extremity,0


In [40]:
train_df = pd.concat([train_df, train_malig], axis=0)
train_df = train_df.reset_index(drop=True)
train_df

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,0
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,0
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,0
...,...,...,...,...,...,...
33267,ISIC_0024302,-1,female,25.0,NaN,1
33268,ISIC_0024303,-1,male,60.0,NaN,1
33269,ISIC_0024304,-1,male,70.0,NaN,1
33270,ISIC_0024305,-1,male,85.0,NaN,1


In [41]:
tfrecord = pd.concat([tfrecord, tfrecord_malig], axis=0)
tfrecord = tfrecord.reset_index(drop=True)

In [42]:
tfrecord

,tfrecord
0,0
1,0
2,6
3,0
4,11
...,...
33267,29
33268,29
33269,29
33270,29


## train_padへの追加

__BGR2GRAY__

In [21]:
#traindf_mv_image_name    :paddingする画像ファイルの名前(tabledataを想定)
#train_folder     :コピー元のtrainのディレクトリ名
#train_agm_folder :padding元のtrain_agmのディレクトリ名
#padfolderName    :train_padに新たに加えるディレクトリ名
def mvPaddingImg(traindf_mv_image_name, train_folder, train_agm_folder, padfolderName):

    shutil.copytree(SIIM_ISIC_Melanoma_Classification + 'pad_jpg/train/' + train_folder,
                    SIIM_ISIC_Melanoma_Classification + 'pad_jpg/train_pad/' + padfolderName)
    
    #train_agm_fileNm = os.listdir(SIIM_ISIC_Melanoma_Classification + 'padjpg/train/' + train_folder)
    for nm in traindf_mv_image_name["image_name"]:
        shutil.copyfile(SIIM_ISIC_Melanoma_Classification + 'pad_jpg/train_agm/' + train_agm_folder + '/' + nm + ".jpg",
                        SIIM_ISIC_Melanoma_Classification + 'pad_jpg/train_pad/' + padfolderName + '/' + nm + ".jpg")
    print("end.")

In [22]:
def initData2020():
    #train(train+2019malig)
    train_malig = pd.read_csv(SIIM_ISIC_Melanoma_Classification + "train_malig_2.csv")
    train_malig = train_malig[train_malig.tfrecord != -1]
    train_malig = train_malig.drop(['diagnosis', 'benign_malignant', 'width', 'height', 'source', 'ext'], axis=1)
    train_malig = train_malig.reset_index(drop=True)
    tfrecord_malig = train_malig.loc[:,['tfrecord']]
    tfrecord_malig = tfrecord_malig-15
    #train_malig = train_malig.drop(['tfrecord'], axis=1)
    
    train_df = pd.read_csv(SIIM_ISIC_Melanoma_Classification + 'train.csv')
    train_df = train_df.drop(['diagnosis','benign_malignant'], axis=1)
    train_df = train_df[train_df.tfrecord != -1]
    train_df = train_df.drop(['width','height','patient_code'], axis=1)

    train_df = train_df.reset_index(drop=True)
    tfrecord = train_df.loc[:,['tfrecord']]
    #train_df = train_df.drop(['tfrecord'], axis=1)
    
    train_df = pd.concat([train_df, train_malig], axis=0)
    train_df = train_df.reset_index(drop=True)
    tfrecord = pd.concat([tfrecord, tfrecord_malig], axis=0)
    tfrecord = tfrecord.reset_index(drop=True)
    
    #test
    test_df = pd.read_csv(SIIM_ISIC_Melanoma_Classification + 'test.csv')
    
    return train_df, test_df, tfrecord

In [25]:
train_df, test_df, tfrecord = initData2020()

In [26]:
train_df

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,target,tfrecord
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,0,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,0,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,0,6
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,0,0
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,0,11
...,...,...,...,...,...,...,...
33267,ISIC_0024302,-1,female,25.0,NaN,1,29
33268,ISIC_0024303,-1,male,60.0,NaN,1,29
33269,ISIC_0024304,-1,male,70.0,NaN,1,29
33270,ISIC_0024305,-1,male,85.0,NaN,1,29


In [27]:
train_df = train_df[train_df.target == 1]
train_df = train_df.reset_index(drop=True)
train_df

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,target,tfrecord
0,ISIC_0149568,IP_0962375,female,55.0,upper extremity,1,0
1,ISIC_0188432,IP_0135517,female,50.0,upper extremity,1,5
2,ISIC_0207268,IP_7735373,male,55.0,torso,1,3
3,ISIC_0232101,IP_8349964,male,65.0,torso,1,9
4,ISIC_0247330,IP_3232631,female,65.0,lower extremity,1,10
...,...,...,...,...,...,...,...
1156,ISIC_0024302,-1,female,25.0,NaN,1,29
1157,ISIC_0024303,-1,male,60.0,NaN,1,29
1158,ISIC_0024304,-1,male,70.0,NaN,1,29
1159,ISIC_0024305,-1,male,85.0,NaN,1,29


In [28]:
train_df[['image_name']] = "GR2GRAY_" + train_df[['image_name']]
train_df

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,target,tfrecord
0,GR2GRAY_ISIC_0149568,IP_0962375,female,55.0,upper extremity,1,0
1,GR2GRAY_ISIC_0188432,IP_0135517,female,50.0,upper extremity,1,5
2,GR2GRAY_ISIC_0207268,IP_7735373,male,55.0,torso,1,3
3,GR2GRAY_ISIC_0232101,IP_8349964,male,65.0,torso,1,9
4,GR2GRAY_ISIC_0247330,IP_3232631,female,65.0,lower extremity,1,10
...,...,...,...,...,...,...,...
1156,GR2GRAY_ISIC_0024302,-1,female,25.0,NaN,1,29
1157,GR2GRAY_ISIC_0024303,-1,male,60.0,NaN,1,29
1158,GR2GRAY_ISIC_0024304,-1,male,70.0,NaN,1,29
1159,GR2GRAY_ISIC_0024305,-1,male,85.0,NaN,1,29


In [29]:
#mv_image_name    :paddingする画像ファイルの名前(tabledataの"image_name"のseriesを想定)
#train_agm_folder :padding元のtrain_agmのディレクトリ名
#padfolderName    :train_padに新たに加えるディレクトリ名
train_agm_folder = "128_128_merge_BGR2GRAY"
padfolderName = "PADDING_" + train_agm_folder + "(malignantOnly)"
mvPaddingImg(train_df, train_agm_folder, padfolderName)

#mv_image_name    :paddingする画像ファイルの名前(tabledataの"image_name"のseriesを想定)
#train_agm_folder :padding元のtrain_agmのディレクトリ名
#padfolderName    :train_padに新たに加えるディレクトリ名
train_agm_folder = "256_256_merge_BGR2GRAY"
padfolderName = "PADDING_" + train_agm_folder + "(malignantOnly)"
mvPaddingImg(train_df, train_agm_folder, padfolderName)

end.
end.


In [23]:
train_df, test_df, tfrecord = initData2020()
train_df = train_df[train_df.target == 1]
train_df = train_df.reset_index(drop=True)

train_df[['image_name']] = "ROTATE_90_" + train_df[['image_name']]

#mv_image_name    :paddingする画像ファイルの名前(tabledataの"image_name"のseriesを想定)
#train_agm_folder :padding元のtrain_agmのディレクトリ名
#padfolderName    :train_padに新たに加えるディレクトリ名
train_folder = "64_64_merge"
train_agm_folder = "64_64_merge_ROTATE_90"
padfolderName = "PADDING_" + train_agm_folder + "(malignantOnly)"
mvPaddingImg(train_df, train_folder, train_agm_folder, padfolderName)

#mv_image_name    :paddingする画像ファイルの名前(tabledataの"image_name"のseriesを想定)
#train_agm_folder :padding元のtrain_agmのディレクトリ名
#padfolderName    :train_padに新たに加えるディレクトリ名
train_folder = "128_128_merge"
train_agm_folder = "128_128_merge_ROTATE_90"
padfolderName = "PADDING_" + train_agm_folder + "(malignantOnly)"
mvPaddingImg(train_df, train_folder, train_agm_folder, padfolderName)

#mv_image_name    :paddingする画像ファイルの名前(tabledataの"image_name"のseriesを想定)
#train_agm_folder :padding元のtrain_agmのディレクトリ名
#padfolderName    :train_padに新たに加えるディレクトリ名
train_folder = "256_256_merge"
train_agm_folder = "256_256_merge_ROTATE_90"
padfolderName = "PADDING_" + train_agm_folder + "(malignantOnly)"
mvPaddingImg(train_df, train_folder, train_agm_folder, padfolderName)

end.
end.
end.


In [24]:
train_df, test_df, tfrecord = initData2020()
train_df = train_df[train_df.target == 1]
train_df = train_df.reset_index(drop=True)

train_df[['image_name']] = "ROTATE_225_" + train_df[['image_name']]

#mv_image_name    :paddingする画像ファイルの名前(tabledataの"image_name"のseriesを想定)
#train_agm_folder :padding元のtrain_agmのディレクトリ名
#padfolderName    :train_padに新たに加えるディレクトリ名
train_folder = "64_64_merge"
train_agm_folder = "64_64_merge_ROTATE_225"
padfolderName = "PADDING_" + train_agm_folder + "(malignantOnly)"
mvPaddingImg(train_df, train_folder, train_agm_folder, padfolderName)

#mv_image_name    :paddingする画像ファイルの名前(tabledataの"image_name"のseriesを想定)
#train_agm_folder :padding元のtrain_agmのディレクトリ名
#padfolderName    :train_padに新たに加えるディレクトリ名
train_folder = "128_128_merge"
train_agm_folder = "128_128_merge_ROTATE_225"
padfolderName = "PADDING_" + train_agm_folder + "(malignantOnly)"
mvPaddingImg(train_df, train_folder, train_agm_folder, padfolderName)

#mv_image_name    :paddingする画像ファイルの名前(tabledataの"image_name"のseriesを想定)
#train_agm_folder :padding元のtrain_agmのディレクトリ名
#padfolderName    :train_padに新たに加えるディレクトリ名
train_folder = "256_256_merge"
train_agm_folder = "256_256_merge_ROTATE_225"
padfolderName = "PADDING_" + train_agm_folder + "(malignantOnly)"
mvPaddingImg(train_df, train_folder, train_agm_folder, padfolderName)

end.
end.
end.


In [ ]:
#addPre           :paddingする画像ファイルの頭に加えるprefix
#mv_image_name    :paddingする画像ファイルの名前(tabledataの"image_name"のseriesを想定)
#train_agm_folder :padding元のtrain_agmのディレクトリ名
#padfolderName    :train_padに新たに加えるディレクトリ名
addPre = "GR2GRAY"
train_agm_folder = "64_64_merge_BGR2GRAY"
padfolderName = "PADDING_" + train_agm_folder + "(malignantOnly)"
mvPaddingImg(addPre, train_df, train_agm_folder, padfolderName)

In [14]:
def initData2020():
    #train(train+2019malig)
    train_malig = pd.read_csv(SIIM_ISIC_Melanoma_Classification + "train_malig_2.csv")
    train_malig = train_malig[train_malig.tfrecord != -1]
    train_malig = train_malig.drop(['diagnosis', 'benign_malignant', 'width', 'height', 'source', 'ext'], axis=1)
    train_malig = train_malig.reset_index(drop=True)
    tfrecord_malig = train_malig.loc[:,['tfrecord']]
    tfrecord_malig = tfrecord_malig-15
    
    
    train_df = pd.read_csv(SIIM_ISIC_Melanoma_Classification + 'train.csv')
    train_df = train_df.drop(['diagnosis','benign_malignant'], axis=1)
    train_df = train_df[train_df.tfrecord != -1]
    train_df = train_df.drop(['width','height','patient_code'], axis=1)
    train_df = train_df.reset_index(drop=True)
    tfrecord = train_df.loc[:,['tfrecord']]
    
    #concate(train+2019malig)
    train_df = pd.concat([train_df, train_malig], axis=0)
    train_df = train_df.reset_index(drop=True)
    
    #padding BGR2GRAY(malignantOnly)
    train_pad = train_df[train_df.target == 1]
    train_pad = train_pad.reset_index(drop=True)
    train_pad[['image_name']] = "GR2GRAY_" + train_pad[['image_name']]
    tfrecord_pad = train_pad.loc[:,['tfrecord']]
    
    #concate(train+2019malig) + BGR2GRAY(malignantOnly)
    train_df = pd.concat([train_df, train_pad], axis=0)
    train_df = train_df.drop(['tfrecord'], axis=1)
    train_df = train_df.reset_index(drop=True)
    tfrecord = pd.concat([tfrecord, tfrecord_malig, tfrecord_pad], axis=0)
    tfrecord = tfrecord.reset_index(drop=True)
    
    
    #test
    test_df = pd.read_csv(SIIM_ISIC_Melanoma_Classification + 'test.csv')
    
    return train_df, test_df, tfrecord

In [15]:
train_df, test_df, tfrecord = initData2020()

In [16]:
train_df

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,0
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,0
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,0
...,...,...,...,...,...,...
34428,GR2GRAY_ISIC_0024302,-1,female,25.0,NaN,1
34429,GR2GRAY_ISIC_0024303,-1,male,60.0,NaN,1
34430,GR2GRAY_ISIC_0024304,-1,male,70.0,NaN,1
34431,GR2GRAY_ISIC_0024305,-1,male,85.0,NaN,1
